<center><span style="font-size:30px">Airbus Ship Detection</span></center>

Shipping traffic is growing fast. More ships increase the chances of infractions at sea like environmentally devastating ship accidents, piracy, illegal fishing, drug trafficking, and illegal cargo movement. This has compelled many organizations, from environmental protection agencies to insurance companies and national government authorities, to have a closer watch over the open seas.<br>

Airbus offers comprehensive maritime monitoring services by building a meaningful solution for wide coverage, fine details, intensive monitoring, premium reactivity and interpretation response. Combining its proprietary-data with highly-trained analysts, they help to support the maritime industry to increase knowledge, anticipate threats, trigger alerts, and improve efficiency at sea.

A lot of work has been done over the last 10 years to automatically extract objects from satellite images with significative results but no effective operational effects. Now Airbus would like to increase the accuracy and speed of automatic ship detection.

To achieve this goal Airbus provides a folder 'train_v2' with 192556 aerial coloured photos  where, in some cases, there are several ships.These photos have an identical size of 768 pixels  x 768 pixels.

Airbus also provides a csv file where there are annotations of previous ships positions using a special encoding. In the arrays (mathematical represention of the photos), every pixel has been given an index number taking the top left pixel as a beginning (first row and first column of the array).<br> 

After that initialisation, we are going over all the array, row by row adding one to the index number of the pixels. That means we have index 1 in the top left pixel and 589824 (768x768) and the bottom right pixel.In case that pixel belongs to a ship a value of 1 will be given to the related array value, conversely if the pixel does not belong to a ship a value of 0 will be given to the related array value. 

The goal of this challenge is to give the best prediction for ships position in new photos. 

In image cases, deep learning algorithms offer quite good solutions by applying convolutional networks. There are basically 2 kind of networks suitable for this detection:

.detection network (SSD,Faster RCN)<br>
.segmentation network (Unet,FCN)<br><br>
Detection networks are usually exploiting the use of a pretrained networks which have been trained previously by standard images where several varied objects are present. In the case of plane or satellite images, it is more difficult to benefit of those standard images. From a sky view, ships are seen as small blobs and pretrained features are not useful.

Segmentation networks may also use pretrained networks, however some of them do not need them. Particularly there is one segmentation network which gives  good results with few image samples. This network is called Unet and is divided in 2 parts, encoder and decoder sub networks.

Encoder Sub Network extracts features from images keeping at each stage the main relevant pixels. Decoder Sub Networks reconstructs original images by recovering original context associated with features extracted previously. In Unet in each step of Encoder Nerwork is connected directly to same level step in Decoder Sub Network.

The main drawbacks of these neural networks are the need of a quick processor as we have a lot of parameters to calculate as well as big RAM memory to store them temporarily.

In my case I have made an first attempt to use my PC with a simple Unet network and a small amount of photos. With my CPU it took about 1 hour to achieve 1 epoch (1 iteration of overall network). With Graphics GPU, it seemed to work faster however I got an out Of Memory error due to 2GB RAM limitation associated to GPU. As memory could not be shared by CPU and GPU in tensorflow, I took the option of Google Collaboratory where I can use GPU with a maximum of 12GB RAM.

The main problem of this solution is that there is not enough hardrive Memory in Google Collaboratory to store all the initial images (26GB). So a trade will be achieved and the first part of the processing will be performed in Jupyter Notebook in my own PC and then the images selected for validation and test will be upload to Google Collaboratory.

Before applying images to segmentation neural networks, a preclassification network will be introduced to check if the final results can be improved. If the preclassification does not improve our results, it will be discarded and only the segmentation network will remain.

<span style="font-size:20px">Data Preprocessing</span>

Before beginning of data preprocessing, 3 folders will be dowloaded from Kaggle platform: __train_v2__,__test_v2__ and __train_ship_segmentations_v2.csv__.

- __train_v2__ folder includes all the images that can be applied to our segmentation neural network (26GB) <br>
- __test_v2__ folder includes all the images that will be used to generate annotations(2GB)<br>
- __train_ship_segmentations_v2.csv__ folder includes a csv file with annotations of all images of __train_v2__ folder

In [1]:
#import all the python libraries required 
import numpy as np 
import pandas as pd 
import os
import shutil

In [2]:
#reading annotations: first column is image file name and second column is EncodedPixels 
#EncodedPixels is a string where ships position is codified in image related mask
#In that mask pixels have only 2 values (0 or 1).
#A pixel with value of 1 belongs to a ship in the original image, a pixel with value 0 does not 
#belong to a ship in the original image
#run length encoding is made of pairs 'position1 length1 position2 length2 ......positionN lengthN'  
#pixels who are in the range [positionI,positionI+lengthI] have value 1
masks = pd.read_csv('../Downloads/train_ship_segmentations_v2.csv/train_ship_segmentations_v2.csv')
masks.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [3]:
#dataframe 'ships' with images with one or more ships
ships=masks.copy()
ships.dropna(inplace=True)
#dataframe 'noships' with images with no ships
noships=masks.copy()
noships = noships[pd.isnull(noships['EncodedPixels'])]

print('There are {}  images with one or more ships'.format(len(ships['ImageId'].unique())))
print('There are {} images with one or more ships'.format(len(noships['ImageId'].unique())))

There are 42556  images with one or more ships
There are 150000 images with one or more ships


In [4]:
# Run length encoding codes each ship in one line of the csv file. So if there are 3 ships in the
#image, there will be 3 lines with the same ImageId and different EncodedPixels features

#grouping in one line all the EncodedPixels features corresponding to the same ImageId
ships['EncodedPixels']= ships['EncodedPixels'].apply(lambda x: x+' ')
ships=ships.groupby(['ImageId'],as_index=False).sum()
ships['EncodedPixels']= ships['EncodedPixels'].apply(lambda x: x.rstrip())
#insertion of a new feature 'ship' which value 1 means there is some ships and value 0 means
#there is no ships
masks['ship'] = masks['EncodedPixels'].apply(lambda x: 1 if isinstance(x, str) else 0)
#if images have one or more ships, 'ship' features will be transform in the number of ships
# in that image
sum_ships=masks[masks['ship']==1].groupby(['ImageId'],as_index=False).agg({'ship': 'sum'})
ships['ship']=sum_ships['ship']
ships['ship']=pd.to_numeric(ships['ship'])

masks.drop(['ship'],axis=1,inplace=True)
#new feature 'size' will be added to show image size in bytes
ships['size']=ships['ImageId'].apply(lambda x: os.path.getsize('../Downloads/train_v2/'+x))

noships['ship']=0
noships['size']=noships['ImageId'].apply(lambda x: os.path.getsize('../Downloads/train_v2/'+x))
#dataframe 'allships' groups images with ships and no ships with additional features 'ship'
#and size
all_ships=pd.concat([ships,noships],axis=0)
#after concatenation the position of images with and without ships will be shuffled
all_ships = all_ships.sample(frac=1).reset_index(drop=True)

In [5]:
all_ships.head()

,ImageId,EncodedPixels,ship,size
0,3d0f061ab.jpg,NaN,0,199102
1,d3590376c.jpg,NaN,0,93351
2,f67d52251.jpg,NaN,0,181992
3,6a0b1741e.jpg,188305 2 189071 4 189836 8 190602 10 191367 13...,1,129323
4,bcfd35f61.jpg,NaN,0,95703


'size' feature has been added to separate high resolution images from low resolution images. This separation can help to detect possible bad images which could lead to an erroneous ship detection results. If we choose a threshold of 45000 bytes, we can notice that low resolution images are very few (553) and they are mainly images without ships (449) with a background color closed to black. In low resolution images with ships we do not see any corruption so this feature will be discarded without deleting any of the images.

In [6]:
#low resolution images number
len(all_ships[all_ships['size']<45000])

553

In [7]:
#low resolution images without ships
low_resolution=all_ships[all_ships['size']<45000]
low_resolution[low_resolution['ship']==0].count()

ImageId          449
EncodedPixels      0
ship             449
size             449
dtype: int64

We can notice that if we separate the number of images by the number of ships included in each image there is a high unbalanced between images with and without ships. So an undersampling process will be performed mainly concerning images without ships(150000). 

In [8]:
all_ships['ship'].value_counts()

0     150000
1      27104
2       7674
3       2954
4       1622
5        925
6        657
7        406
8        318
9        243
10       168
11       144
12       124
14        76
13        75
15        66
Name: ship, dtype: int64

We are going to discard a big amount of photos because in Google Collaboratory, even with a 12GB maximum memory, memory limit can be reach very fastly when complex neural networks are trained. So,about a maximum of 4000 images will be chosen to be uploaded to Google Collaboratory. If the number of ships in each image is considered as the value of a class, a maximum number of images per class will be selected. In this case there are 15 classes and as beyond class '7' all the classes have less than 400 images, a maximim 400 images per class will be decided.

In [9]:
#the downsampling will be performed by sample function. 400 images maximum will be chosen by class
# in practice only images number with 7 ships or less ships will be reduced
MAX_IMAGES_PER_CLASS=400

sample_balanced=all_ships.groupby('ship').apply(lambda x: x.sample(MAX_IMAGES_PER_CLASS) if len(x)>MAX_IMAGES_PER_CLASS else x)

In [10]:
#sample_balanced is the dataframe that will be chosen to upload images to Google Collaboratory
sample_balanced['ship'].value_counts().sort_index()

0     400
1     400
2     400
3     400
4     400
5     400
6     400
7     400
8     318
9     243
10    168
11    144
12    124
13     75
14     76
15     66
Name: ship, dtype: int64

In [11]:
sample_balanced.count()

ImageId          4414
EncodedPixels    4014
ship             4414
size             4414
dtype: int64

In [12]:
#'size' feature is eliminated 
sample_balanced.drop(['size'],axis=1,inplace=True)
#rows in this dataframe are shuffled
sample_balanced = sample_balanced.sample(frac=1).reset_index(drop=True)

sample_balanced.head()

,ImageId,EncodedPixels,ship
0,e2e6c2dc5.jpg,246392 1 247159 3 247926 5 248693 7 249460 9 2...,1
1,51f397469.jpg,381587 1 382353 4 383120 5 383887 7 384654 9 3...,7
2,46507a0d0.jpg,32295 11 33063 30 33831 38 34598 42 35366 42 3...,14
3,a44b42d54.jpg,499801 1 500568 3 501335 5 502102 7 502869 9 5...,8
4,444e9617e.jpg,302422 2 303188 4 303954 7 304721 8 305489 9 3...,3


In [ ]:
# from sample_balanced dataframe we are going to copy corresponding images
# into airbus_ship_images_4414 folder which will be used in Google Collaboratory 
for index,row in sample_balanced.iterrows():
    newPath = shutil.copy('../Downloads/train_v2/'+row['ImageId'],'../Downloads/airbus_ship_images_4414/')
#sample_balanced dataframe with previous images annotations will be copied into a csv file 
#to be uploaded to Google Collaboratory 
sample_balanced.to_csv('ships_dataset.csv', sep=',', encoding='utf-8',index=False)
shutil.copy('ships_dataset.csv','../Downloads/')

The criterium for choosing these previous images was the maximum balance between classes. With this criterium, we can expect better results in segmentation neural networks.

Now, as I said at the beginning, we foresee to check if an images preclassification can improve segmentation results. For this preclassification, a balance of images with and without ships is required for good features extraction. 

As in the previous case, due to Google Collaboratory RAM size in neural networks training, a maximum of about 4000 images will be chosen.

In [13]:
#in this case we will select maximum 155 images with ships per class and 2035 images without ships
#the undersampling will be performed by sample function
MAX_IMAGES_SHIP_CLASS=155
MAX_IMAGES_NONSHIP_CLASS=2035
sample_ship=all_ships[all_ships['ship']!=0].groupby('ship').apply(lambda x: x.sample(MAX_IMAGES_SHIP_CLASS) if len(x)>MAX_IMAGES_SHIP_CLASS else x)
sample_nonship=all_ships[all_ships['ship']==0].groupby('ship').apply(lambda x: x.sample(MAX_IMAGES_NONSHIP_CLASS))
sample_class=pd.concat([sample_ship,sample_nonship],axis=0,ignore_index=True)
sample_class = sample_class.sample(frac=1).reset_index(drop=True)
sample_class.drop(['size'],axis=1,inplace=True)
sample_class['ship']=sample_class['ship'].apply(lambda x: 1 if x!=0 else x)
sample_class.head()

,ImageId,EncodedPixels,ship
0,50eddd13d.jpg,257763 3 258531 7 259299 10 260066 11 260834 1...,1
1,71d47dcfb.jpg,224343 3 225105 9 225867 16 226632 16 227400 1...,1
2,f3b41e1f7.jpg,320239 6 321002 11 321770 11 322538 11 323306 ...,1
3,6a7de5958.jpg,521862 2 522630 6 523397 10 524165 14 524933 1...,1
4,accdf6c3e.jpg,63052 1 63820 2 64588 4 65357 4 66125 5 66894 ...,1


In [14]:
print("Images with    ships ",len(sample_class[sample_class['ship']==1]))
print("Images without ships ",len(sample_class[sample_class['ship']==1]))

Images with    ships  2035
Images without ships  2035


In [ ]:
# from sample_class dataframe we are going to copy corresponding images
# into airbus_ship_images_4070 folder which will be used in Google Collaboratory 
for index,row in sample_class.iterrows():
    newPath = shutil.copy('../Downloads/train_v2/'+row['ImageId'],'../Downloads/airbus_ship_images_4070/')
#sample_class dataframe with previous images annotations will be copied into a csv file 
#to be uploaded to Google Collaboratory
sample_class.to_csv('new_ships_dataset.csv', sep=',', encoding='utf-8',index=False)
shutil.copy('new_ships_dataset.csv','../Downloads/')    